In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Check Libraires 

In [ ]:
pip install tensorflow-text==2.6.0-dev20210407


In [ ]:
pip install --upgrade tensorflow-gpu

## Imports

In [ ]:
# Imports
import cv2
import os
from tqdm import tqdm
from glob import glob
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D

## Loading Dataset

In [ ]:
df = pd.read_csv("../input/state-farm-distracted-driver-detection/driver_imgs_list.csv")
df.head()

In [ ]:
# Group bt Drivers/ Test Subjects
by_drivers = df.groupby('subject')

unique_drivers = by_drivers.groups.keys()

print("There are: ", len(unique_drivers), " unique drivers")
print('There is a mean of ',round(by_drivers.count()['classname']), ' images by driver.')

In [ ]:
NUMBER_CLASSES = 10

In [ ]:
# Read with opencv
def get_cv2_image(path, img_rows, img_cols, color_type=3):
    """
    Function that return an opencv image from the path and the right number of dimension
    """
    if color_type == 1: # Loading as Grayscale image
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    elif color_type == 3: # Loading as color image
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Converts to RGB

    img = cv2.resize(img, (img_rows, img_cols)) # Reduce size
    return img

# Loading Training dataset
def load_train(img_rows, img_cols, color_type=3):
    """
    Return train images and train labels from the original path
    """
    train_images = [] 
    train_labels = []
    # Loop over the training folder 
    for classed in tqdm(range(NUMBER_CLASSES)):
        print('Loading directory c{}'.format(classed))
        files = glob(os.path.join('../input/state-farm-distracted-driver-detection/imgs/train/c' + str(classed), '*.jpg'))
        for file in files:
            img = get_cv2_image(file, img_rows, img_cols, color_type)
            train_images.append(img)
            train_labels.append(classed)
    return train_images, train_labels 

def read_and_normalize_train_data(img_rows, img_cols, color_type):
    """
    Load + categorical + split
    """
    X, labels = load_train(img_rows, img_cols, color_type)
    y = np_utils.to_categorical(labels, 10) #categorical train label
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # split into train and test
    x_train = np.array(x_train, dtype=np.uint8).reshape(-1,img_rows,img_cols,color_type)
    x_test = np.array(x_test, dtype=np.uint8).reshape(-1,img_rows,img_cols,color_type)
    
    return x_train, x_test, y_train, y_test

# Loading validation dataset
def load_test(size=200000, img_rows=64, img_cols=64, color_type=3):
    """
    Same as above but for validation dataset
    """
    path = os.path.join('../input/state-farm-distracted-driver-detection/imgs/test', '*.jpg')
    files = sorted(glob(path))
    X_test, X_test_id = [], []
    total = 0
    files_size = len(files)
    for file in tqdm(files):
        if total >= size or total >= files_size:
            break
        file_base = os.path.basename(file)
        img = get_cv2_image(file, img_rows, img_cols, color_type)
        X_test.append(img)
        X_test_id.append(file_base)
        total += 1
    return X_test, X_test_id

def read_and_normalize_sampled_test_data(size, img_rows, img_cols, color_type=3):
    test_data, test_ids = load_test(size, img_rows, img_cols, color_type)   
    test_data = np.array(test_data, dtype=np.uint8)
    test_data = test_data.reshape(-1,img_rows,img_cols,color_type)
    return test_data, test_ids

In [ ]:
# dimension of images
img_rows = 128 
img_cols = 128

color_type = 1 # grey
nb_test_samples = 200

In [ ]:
# loading train images
x_train, x_test, y_train, y_test = read_and_normalize_train_data(img_rows, img_cols, color_type)

# loading validation images
test_files, test_targets = read_and_normalize_sampled_test_data(nb_test_samples, img_rows, img_cols, color_type)

## EDA

In [ ]:
x_train_size = len(x_train)
x_test_size = len(x_test)
test_files_size = len(np.array(glob(os.path.join('../input/state-farm-distracted-driver-detection/imgs/test', '*.jpg'))))

> ## Statistical numbers about the data

In [ ]:
print('There are %s total images.' %(x_train_size + x_test_size + test_files_size))
print('There are %d total training categories.' %NUMBER_CLASSES )
print('There are %d training images.' % x_train_size)
print('There are %d validation images.' % x_test_size)
print('There are %d test images.'% test_files_size)

> ## Data Visulization

In [ ]:
import plotly.express as px

px.histogram(df, x="classname", color="classname", title="Number of images by categories ")

In [ ]:
# Number of Images by Drivers / Test Subject

In [ ]:
# np.save('./x_train.npy',x_train)
# np.save('./y_train.npy',y_train)
# np.save('./x_test.npy',x_test)
# np.save('./y_test.npy',y_test)

# Fully connected layer

## Fill this to continue running smoothly, I think

In [ ]:
# x_train = np.load('../input/dl-project/x_train.npy').astype('float32')/255
# y_train = np_utils.to_categorical(np.load('../input/dl-project/y_train.npy'))
# x_val = np.load('../input/dl-project/x_test.npy').astype('float32')/255
# y_val = np_utils.to_categorical(np.load('../input/dl-project/y_test.npy'))
no_epoch = 5
batch_size = 64
img_height = img_rows
img_width = img_cols
channels = 1

In [ ]:
# x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)/255
# y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
# x_val = tf.convert_to_tensor(x_val, dtype=tf.float32)/255
# y_val = tf.convert_to_tensor(y_val, dtype=tf.float32)

## Building initial model

In [ ]:
# temp
x_train_FC = x_train.reshape((x_train_size, img_rows*img_cols*1))
x_train_FC = x_train_FC.astype('float32')/255
x_val_FC = x_test.reshape((x_test_size, img_rows*img_cols*1))
x_val_FC = x_val_FC.astype('float32')/255
y_val = y_test

In [ ]:
FC_init = Sequential()
# FC_init.add(Flatten())
FC_init.add(Dense(512, activation='relu', name='Layer_1', input_shape=(img_width * img_height * channels,)))
FC_init.add(Dense(256, activation='relu', name='Layer_2'))
FC_init.add(Dense(128, activation='relu', name='Layer_3'))
FC_init.add(Dense(10, activation='softmax'))

In [ ]:
opt = keras.optimizers.Adam(learning_rate=10e-3)
FC_init.compile(optimizer=opt,
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
FC_init.summary()

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=2,
    verbose=1,
    mode='min',
    baseline=None,
    restore_best_weights=True
)

In [ ]:
History = FC_init.fit(x_train_FC,y_train, validation_data=(x_val_FC,y_val), verbose = 1, epochs = no_epoch, batch_size = batch_size,callbacks=[early_stopping])

In [ ]:
acc = History.history['accuracy']
val_acc = History.history['val_accuracy']
loss = History.history['loss']
val_loss = History.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## Predict 

In [ ]:
test_files.dtype

In [ ]:
test_files[0].shape

In [ ]:
x_train.dtype

In [ ]:
print(len(test_files))
print(test_files_size)


In [ ]:
test_imgs = test_files.reshape((nb_test_samples, img_rows*img_cols*1))
test_imgs = test_imgs.astype('float32')/255


In [ ]:
pred = FC_init.predict(test_imgs)
pred[0]

In [ ]:
FC_init.evaluate(test_imgs, pred)

### Evaluate the model

## Building Baseline CNN Model

In [ ]:
# temp
x_train_CNN = x_train.reshape((x_train_size, img_rows,img_cols,1))
x_train_CNN = x_train_CNN.astype('float32')/255
x_val_CNN = x_test.reshape((x_test_size, img_rows,img_cols,1))
x_val_CNN = x_val_CNN.astype('float32')/255
y_val = y_test

In [ ]:
CNN_model = Sequential()
CNN_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, channels)))
CNN_model.add(MaxPooling2D((2, 2)))
CNN_model.add(Conv2D(64, (3, 3), activation='relu'))
CNN_model.add(MaxPooling2D((2, 2)))
CNN_model.add(Conv2D(64, (3, 3), activation='relu'))
CNN_model.add(Flatten())
CNN_model.add(Dense(64, activation='relu'))
CNN_model.add(Dense(10, activation='softmax'))


In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)
CNN_model.compile(optimizer="rmsprop",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
CNN_model.summary()

In [ ]:
History_CNN = CNN_model.fit(x_train_CNN,y_train, validation_data=(x_val_CNN,y_val), verbose = 1, epochs = no_epoch, batch_size = batch_size,callbacks=[early_stopping])

In [ ]:
acc = History_CNN.history['accuracy']
val_acc = History_CNN.history['val_accuracy']
loss = History_CNN.history['loss']
val_loss = History_CNN.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
test_imgs = test_files.reshape((nb_test_samples, img_rows,img_cols,1))
test_imgs = test_imgs.astype('float32')/255


In [ ]:
pred = CNN_model.predict(test_imgs)
pred[0]

## Data Augmentation

In [ ]:
Traindatagen = ImageDataGenerator(
      featurewise_center = True,
      featurewise_std_normalization = True,
      rescale = 1.0/255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      vertical_flip=False,
      fill_mode='nearest')


Valdatagen =  ImageDataGenerator(featurewise_center = True, 
                                 featurewise_std_normalization = True, 
                                 rescale=1.0/ 255, 
                                 validation_split = 0.2)

In [ ]:
# train_generator = Traindatagen.flow(x_train_CNN, y_train, batch_size = 256)
# val_generator = Valdatagen.flow(x_val_CNN, y_val, batch_size = 64)

# Transfer learning using VGG16

In [ ]:
# loading train images
x_train_vgg, x_test_vgg, y_train_vgg, y_test_vgg = read_and_normalize_train_data(img_rows, img_cols, 3)

# loading validation images
# test_files, test_targets = read_and_normalize_sampled_test_data(nb_test_samples, img_rows, img_cols, 3)

In [ ]:
train_generator = Traindatagen.flow(x_train_vgg, y_train_vgg, batch_size = 256)
val_generator = Valdatagen.flow(x_test_vgg, y_test_vgg, batch_size = 64)

In [ ]:
# resnet_rs50_conv_base = tf.keras.applications.resnet50.ResNet50(
#     include_top=False,
#     weights='imagenet',
#     input_shape=(128,128,3))

from keras.applications.vgg16 import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(128, 128, 3))

In [ ]:
# resnet_rs50_conv_base.trainable = False
conv_base.trainable = False

In [ ]:
# resnetRs = Sequential()
# # resnetRs.add(resnet_rs50_conv_base)
# resnetRs.add(conv_base)
# resnetRs.add(Flatten())
# resnetRs.add(Dense(256, activation='relu'))
# resnetRs.add(Dense(10, activation='softmax'))

In [ ]:
vgg_model = Sequential()
# # resnetRs.add(resnet_rs50_conv_base)
vgg_model.add(conv_base)
vgg_model.add(Flatten())
vgg_model.add(Dense(256, activation='relu'))
vgg_model.add(Dense(10, activation='softmax'))

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)
vgg_model.compile(optimizer=adam,
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
vgg_model.summary()

In [ ]:
epochs = 20
train_bs = 256
valdi_bs = 64

In [ ]:
history = vgg_model.fit(train_generator,
                          validation_data=val_generator,
                          steps_per_epoch=len(x_train_vgg) // train_bs, 
                          epochs=epochs,
                          validation_steps =len(x_test_vgg)//valdi_bs, 
                          verbose = 1)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

### Model

In [ ]:
def classificationModel():
    inp = keras.layers.Input(shape=(128, 128, 3))
    vgg = VGG16(weights='imagenet',
                  include_top=False,
                  input_tensor = inp,
                  input_shape=(128, 128, 3))
    vgg.trainable = False
    
    x = vgg.get_layer('block5_pool').output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(256, activation='relu')(x)
    output = keras.layers.Dense(10, activation='softmax')(x)
    
    model = tf.keras.models.Model(inputs = inp, outputs=output)

    return model
    

In [ ]:
keras.backend.clear_session()
model = classificationModel()
model.summary()

In [ ]:
# model = Sequential()
# # resnetRs.add(resnet_rs50_conv_base)
# model.add(conv_base)
# model.add(Flatten())
# model.add(Dense(256, activation='relu'))
# model.add(Dense(10, activation='softmax'))

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
epochs = 20
train_bs = 256
valdi_bs = 64

## CNN Viualization

In [ ]:
%%capture
!pip install wandb

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

In [ ]:
class GradCAM:
    """
    Reference:
        https://www.pyimagesearch.com/2020/03/09/grad-cam-visualize-class-activation-maps-with-keras-tensorflow-and-deep-learning/
    """

    def __init__(self, model, layerName):
        self.model = model
        self.layerName = layerName

        self.gradModel = tf.keras.models.Model(inputs=[self.model.inputs],
                                               outputs=[self.model.get_layer(self.layerName).output, self.model.output])

    def compute_heatmap(self, image, classIdx, eps=1e-8):
        with tf.GradientTape() as tape:
            tape.watch(self.gradModel.get_layer(self.layerName).variables)
            inputs = tf.cast(image, tf.float32)
            (convOutputs, predictions) = self.gradModel(inputs)

            if len(predictions) == 1:
                # Binary Classification
                loss = predictions[0]
            else:
                loss = predictions[:, classIdx]

        grads = tape.gradient(loss, convOutputs)

        castConvOutputs = tf.cast(convOutputs > 0, "float32")
        castGrads = tf.cast(grads > 0, "float32")
        guidedGrads = castConvOutputs * castGrads * grads

        convOutputs = convOutputs[0]
        guidedGrads = guidedGrads[0]

        weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
        cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)

        (w, h) = (image.shape[2], image.shape[1])
        heatmap = cv2.resize(cam.numpy(), (w, h))

        numer = heatmap - np.min(heatmap)
        denom = (heatmap.max() - heatmap.min()) + eps
        heatmap = numer / denom
        heatmap = (heatmap * 255).astype("uint8")

        return heatmap


    def overlay_heatmap(self, heatmap, image, alpha=0.5, colormap=cv2.COLORMAP_HOT):
        heatmap = cv2.applyColorMap(heatmap, colormap)
        output = cv2.addWeighted(image, alpha, heatmap, 1 - alpha, 0)

        return (heatmap, output)

In [ ]:
class GRADCamLogger(tf.keras.callbacks.Callback):
    def __init__(self, validation_data, layer_name):
      super(GRADCamLogger, self).__init__()
      self.validation_data = validation_data
      self.layer_name = layer_name

    def on_epoch_end(self, logs, epoch):
      images = []
      grad_cam = []

      ## Initialize GRADCam Class
      cam = GradCAM(model, self.layer_name)

      for image in self.validation_data:
        image = np.expand_dims(image, 0)
        pred = model.predict(image)
        classIDx = np.argmax(pred[0])
  
        ## Compute Heatmap
        heatmap = cam.compute_heatmap(image, classIDx)
        
        image = image.reshape(image.shape[1:])
        image = image*255
        image = image.astype(np.uint8)

        ## Overlay heatmap on original image
        heatmap = cv2.resize(heatmap, (image.shape[0],image.shape[1]))
        (heatmap, output) = cam.overlay_heatmap(heatmap, image, alpha=0.5)

        images.append(image)
        grad_cam.append(output)

      wandb.log({"images": [wandb.Image(image)
                            for image in images]})
      wandb.log({"gradcam": [wandb.Image(cam)
                            for cam in grad_cam]})

In [ ]:
## Prepare sample images to run your GradCam on. 
sample_images, sample_labels = val_generator[20]
sample_images.shape, sample_labels.shape

In [ ]:
# wandb.init(entity='ayush-thakur', project='interpretability')
wandb.init(project="test", entity="team-7")

In [ ]:
# History_Rs = resnetRs.fit(train_generator,
#          validation_data=val_generator,
#          steps_per_epoch=len(x_train_rs) // train_bs, epochs=epochs,
#           validation_steps =len(x_test_rs)//valdi_bs, verbose = 1)


history = model.fit(train_generator,
                          validation_data=val_generator,
                          steps_per_epoch=len(x_train_rs) // train_bs, 
                          epochs=epochs,
                          validation_steps =len(x_test_rs)//valdi_bs, 
                          verbose = 1,
                          callbacks=[WandbCallback(data_type="image", validation_data=(sample_images, sample_labels)),
                               GRADCamLogger(sample_images, layer_name='block5_conv3')])


In [ ]:
# History_Rs_2 = resnetRs.fit(train_generator,
#          validation_data=val_generator,
#          steps_per_epoch=len(x_train_rs) // train_bs, epochs=10,
#           validation_steps =len(x_test_rs)//valdi_bs, verbose = 1)

In [ ]:
resnetRs.save('./my_model.h5')

In [ ]:
new_model = tf.keras.models.load_model('./my_model.h5')

In [ ]:
acc = History_Rs_2.history['accuracy']
val_acc = History_Rs_2.history['val_accuracy']
loss = History_Rs_2.history['loss']
val_loss = History_Rs_2.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()